Notebook follows the PyTorch tutorial at <https://www.youtube.com/watch?v=GIsg-ZUy0MY>. At timestamp 0:00:00

In [1]:
import torch

# Tensors

A tensor is a number, vector, matrix or any kind of n-dimensional array.

In [2]:
t1 = torch.tensor(4.)
t1

tensor(4.)

In [3]:
t1.dtype

torch.float32

In [4]:
t2 = torch.tensor([1., 2, 3, 4])
t2

tensor([1., 2., 3., 4.])

In [5]:
t3 = torch.tensor([[5., 6], [7, 8], [9, 10]])
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [6]:
t4 = torch.tensor([
    [[11, 12, 13],
    [14, 15, 16]],
    [[17, 18, 19],
    [20, 21, 22]]
])
t4

tensor([[[11, 12, 13],
         [14, 15, 16]],

        [[17, 18, 19],
         [20, 21, 22]]])

Tensors can have any number of dimensions and different lengths across the dimensions. We can inspect the length along each dimension using the ```.shape``` property of a tensor.

In [7]:
t1.shape

torch.Size([])

In [8]:
t2.shape

torch.Size([4])

In [9]:
t3.shape

torch.Size([3, 2])

In [10]:
t4.shape

torch.Size([2, 2, 3])

# Tensor operations and gradients

We can combine tensors with the usual arithmetic operations. For example:

In [11]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)

We'll create ```y``` which is a new tensor combining all 3:

In [12]:
y = w * x + b
y

tensor(17., grad_fn=<AddBackward0>)

As expected `y` is a tensor with the value $ 3 \times 4 + 5 = 17$ . What makes Pytorch special is that we can **automatically** compute the derivative of `y` with respect to the tensors that have `requires_grad = True` i.e. `w` and `b`. To compute the derivatives we can call the `.backward` method on `y`. 

In [13]:
y.backward()

The derivates of `w` and `b` are stored in their `.grad` properties.

In [14]:
print(f'dy/dx: {y.grad}') # Should be None since requires_grad for x was not set to True
print(f'dy/dw: {w.grad}') # dy/dw = x which is equal to 3 from their initialization
print(f'dy/db: {b.grad}') # dy/db = 1

dy/dx: None
dy/dw: 3.0
dy/db: 1.0


<ipython-input-14-7c5a7d49f033>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print(f'dy/dx: {y.grad}') # Should be None since requires_grad for x was not set to True


# Interoperability with Numpy

In [15]:
import numpy as np

x = np.array([[1, 2], [3, 4.]])
x

array([[1., 2.],
       [3., 4.]])

## Converting to tensor

There a two methods

### Using `.from_numpy()`

This does not create a copy of the numpy array and uses the same space in memory.

In [16]:
y = torch.from_numpy(x)
y

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

### Using `.tensor()`

This creates a copy of the numpy array `x`.

In [17]:
y = torch.tensor(x)
y

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

In [18]:
x.dtype, y.dtype

(dtype('float64'), torch.float64)

We can convert a PyTorch tensor to a Numpy array using the `.numpy()` method of a tensor.

In [19]:
z = y.numpy()
z

array([[1., 2.],
       [3., 4.]])

This interoperability is important bc most datasets are read and preprocessed as Numpy arrays. Also predictions have to be converted back to Numpy arrays.

# Exercises

## 1. What would it look it x, w and/or b were matrices? What will their gradients look like?

In [28]:
x = torch.tensor([1., 2, 3])
w = torch.tensor([[1., 1, 1], [1, 1, 1], [1, 1, 1]], requires_grad=True)
b = torch.tensor([2, 2., 4], requires_grad=True)

In [29]:
y = w * x + b
y

tensor([[3., 4., 7.],
        [3., 4., 7.],
        [3., 4., 7.]], grad_fn=<AddBackward0>)

In [30]:
y.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

The gradient can only be implicitly created for scalar outputs.

In [23]:
print(f'dy/dx: {y.grad}') # Should be None since requires_grad for x was not set to True
print(f'dy/dw: {w.grad}') # dy/dw = x which is equal to 3 from their initialization
print(f'dy/db: {b.grad}') # dy/db = 1

dy/dx: None
dy/dw: None
dy/db: None


<ipython-input-23-7c5a7d49f033>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print(f'dy/dx: {y.grad}') # Should be None since requires_grad for x was not set to True


## 2. What if we had a chain of equations,
$$
y = w * x + b \\
z = 1 * y + m \\
w = c * z + d \\
$$

What will calling `w.grad` do?

In [24]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)
m = torch.tensor(7., requires_grad=True)
c = torch.tensor(2., requires_grad=True)
d = torch.tensor(6., requires_grad=True)

In [25]:
y = w * x + b
z = 1 * y + m
w = c * z + d
y, z, w

(tensor(17., grad_fn=<AddBackward0>),
 tensor(24., grad_fn=<AddBackward0>),
 tensor(54., grad_fn=<AddBackward0>))

In [26]:
y.backward()

In [27]:
print(f'dy/dx: {y.grad}') # Should be None since requires_grad for x was not set to True
print(f'dy/dw: {w.grad}') # dy/dw = x which is equal to 3 from their initialization
print(f'dy/db: {b.grad}') # dy/db = 1

dy/dx: None
dy/dw: None
dy/db: 1.0


<ipython-input-27-7c5a7d49f033>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print(f'dy/dx: {y.grad}') # Should be None since requires_grad for x was not set to True
<ipython-input-27-7c5a7d49f033>:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print(f'dy/dw: 

So, apparently

```python
{
    "my_name": "Mutia Babila",
    "age": 23
}
me = set([1,2,3])
```